In [7]:
from sdv.demo import load_timeseries_demo
import pandas as pd

In [8]:
data = pd.read_csv('./AAPL Historical Data.csv')

In [9]:
data.head()

,Date,Price
0,11/01/2013,19.86
1,12/01/2013,20.04
2,01/01/2014,17.88
3,02/01/2014,18.79
4,03/01/2014,19.17


In [11]:
context_columns = ['Price']

In [ ]:
entity_columns = ['']

In [12]:
sequence_index = 'Date'

In [13]:
from sdv.timeseries import PAR


In [14]:
model = PAR(
   entity_columns=entity_columns,
   context_columns=context_columns,
   sequence_index=sequence_index,
  )

NameError: name 'entity_columns' is not defined